In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_vertexai import ChatVertexAI
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
llm=ChatVertexAI(model_name="gemini-2.5-pro")

In [4]:
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]

In [ ]:
def chat_node(state:ChatState):
    # take user query 

In [ ]:
graph=StateGraph(ChatState)